In [220]:
import tekore as tk
import asyncio

client_id = '1a11d5c3f04d467da9a1de4331efc7e8'
client_secret = '695877a5bba444fa9b433594f160a094'

## Authorize app

In [221]:
app_token = tk.request_client_token(client_id, client_secret)


In [222]:
spotify = tk.Spotify(app_token, chunked_on=True)

In [223]:
party = {
        "lenar": 'nndt22q5jz94jmno65men0kck',
        "igor": 'pc5gqg4ffd3lebpnqo9q7kxef',
        "roma": 'qni8wxx6xl3e1mm57bq0uch4c',
        "artem": 'q4snwiny17hwum3zg434ien5c',
        "farit": '31rsvq6w2v4hlz7aluauciait62y'
    }

In [224]:
def get_track_features(track, member):
    return {"song_name": track.name,
     "explicit": track.explicit,
     "popularity": track.popularity,
     "member": member
    }
    

In [226]:
tracks_info = {}
audio_features = {}
for member in party:
    print(member)
    for playlist in spotify.playlists(party[member]).items:
        if member=="roma":
            print(playlist)
        items = spotify.playlist_items(playlist.id)
        items = list(spotify.all_items(items))
        tracks_info.update({item.track.id:get_track_features(item.track, member) for item in items})
        features = spotify.tracks_audio_features(list(tracks_info.keys()))
        features = {feature.id:({k:v for k, v in feature.__dict__.items()}) 
                               for feature in features}
        {features[k].update(tracks_info[k]) for k in features}
        audio_features.update(features)
    

lenar
igor
roma
artem
farit


In [228]:
import pandas as pd

df = pd.DataFrame(audio_features.values())

In [229]:
df.to_csv("party_preference.csv", index=False)

## Recommendation system (simple)

In [230]:
import pandas as pd
import numpy as np

data = pd.read_csv("party_preference.csv")
a = data.select_dtypes(np.number)
a.to_csv("song_features.csv", index=False)

In [231]:
data.member.unique()

array(['lenar', 'igor', 'farit', 'artem'], dtype=object)

In [232]:
import csv

def get_party_data():
    party_data = csv.DictReader(
        (x for x in open("party_preference.csv","r"))
    )
    
    return party_data

def get_song_features():
    song_features = csv.DictReader(
            (x for x in open("song_features.csv","r"))
        )
    return song_features

In [233]:
import json
from itertools import islice
for line in islice(get_party_data(), 1):
    print(json.dumps(line, indent=4))

for line in islice(get_song_features(), 1):
    print(json.dumps(line, indent=4))

{
    "id": "3uOIoS7lOvzQAtbbyttHp6",
    "acousticness": "0.0791",
    "analysis_url": "https://api.spotify.com/v1/audio-analysis/3uOIoS7lOvzQAtbbyttHp6",
    "danceability": "0.717",
    "duration_ms": "201160",
    "energy": "0.758",
    "instrumentalness": "7.08e-06",
    "key": "2",
    "liveness": "0.293",
    "loudness": "-5.576",
    "mode": "0",
    "speechiness": "0.268",
    "tempo": "91.983",
    "time_signature": "4",
    "track_href": "https://api.spotify.com/v1/tracks/3uOIoS7lOvzQAtbbyttHp6",
    "type": "audio_features",
    "uri": "spotify:track:3uOIoS7lOvzQAtbbyttHp6",
    "valence": "0.252",
    "song_name": "BUZZCUT (feat. Danny Brown)",
    "explicit": "True",
    "popularity": "58",
    "member": "lenar"
}
{
    "acousticness": "0.0791",
    "danceability": "0.717",
    "duration_ms": "201160",
    "energy": "0.758",
    "instrumentalness": "7.08e-06",
    "key": "2",
    "liveness": "0.293",
    "loudness": "-5.576",
    "mode": "0",
    "speechiness": "0.268",
 

In [234]:
from lightfm.data import Dataset

In [235]:
dataset=Dataset()
dataset.fit(
    (x['member'] for x in get_party_data()),
    (x['id'] for x in get_party_data())
)

In [236]:
num_users, num_items = dataset.interactions_shape()
print('Num users: {}, num_items {}.'.format(num_users, num_items))

Num users: 4, num_items 2298.


In [237]:
dataset.fit_partial(items=(x['id'] for x in get_party_data()),
                    item_features=(tuple(x.values()) for x in get_song_features()))

In [238]:
interactions,_ = dataset.build_interactions((x['member'],x['id']) for x in get_party_data())
repr(interactions)

"<4x2298 sparse matrix of type '<class 'numpy.int32'>'\n\twith 2298 stored elements in COOrdinate format>"

In [239]:
ids = [x['id'] for x in get_party_data()]
item_features = dataset.build_item_features(((x[0], [tuple(x[1].values())])
                                              for x in zip(ids,get_song_features())))


In [321]:
from lightfm import LightFM

model = LightFM(loss='warp-kos', k=3)
model.fit(interactions, item_features=item_features, epochs=100)

In [322]:
from lightfm.evaluation import auc_score

# Compute and print the AUC score
train_auc = auc_score(model, interactions, item_features=item_features, num_threads=4).mean()
print('Collaborative filtering train AUC: %s' % train_auc)

Collaborative filtering train AUC: 0.9498484


In [317]:
from collections import Counter
n_users, n_items = interactions.shape
common_scores = np.zeros(n_items)
simple_rank = Counter()
for user_id in range(n_users):
    scores = model.predict(user_id, np.arange(n_items))
    top_items = data['id'].iloc[np.argsort(-scores)].tolist()[:1000]
    simple_rank+=Counter(top_items)
    common_scores+=scores
    

In [318]:
sorted_scores = np.sort(-scores)
top_items = {k:v for k,v in zip(data['id'].iloc[np.argsort(-scores)].tolist(), sorted_scores)}
simple_rank = {k:top_items[k] for k,v in simple_rank.most_common(50)}

In [319]:
simple_rank = {k:v for k,v in sorted(simple_rank.items(), key=lambda x:x[1])}

In [320]:
for rank in simple_rank:
    print(data[data['id']==rank]['song_name'].values)

['Bolo Ta Ra Ra']
['Paint Me Silver']
['Sentimental Trash']
['Comanche Moon']
['4ÆM']
['Summer Madness']
['Check Yo Self - Remix']
['Seagulls']
['Hey Pachuco!']
['As You Move Through The World']
['Space Echo']
['Electric Feel']
["Where's Your Head At"]
['Cypis - Gdzie jest biały węgorz ? (Zejście)']
["Don't Worry Be Happy"]
['Rock Music']
['Ordinary Pleasure']
['Русский андеграунд']
['xxx']
['Walkabout (W/ Noah Lennox)']
['Feel']
['Мёртвый сезон']
['いつも雨']
['Contact']
['Slow Jam 1']
['funee monkee gif']
['False Mirror']
['Tank!']
['The Getaway']
['Искра']
['Lovesick']
['真夜中のドア〜stay with me']
['Kanzen kankaku Dreamer']
['Mariem']
['ЗАХОТЕЛ – ВЗЯЛ']
['Первомай']
['Soul Tan']
['Lucid Dream']
['Higashikaze ni Sasoware']
['Lovely']
['Atomic Bomb - Cover']
['Нелюбовь']
['Sunflower']
["Lesson In Leavin'"]
['Something In The Way']
['Englishman In New York - My Songs Version']
["Cruisin'"]
['Et cétéra']
['Buttercup']
['Resonance']
